<a href="https://colab.research.google.com/github/junyealim/Homework/blob/main/BPE%20%EB%85%BC%EB%AC%B8_%EC%9A%94%EC%95%BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Neural Machine Translation of Rare Words with Subword Units**

## **Abstract**

*Neural machine translation (NMT)* 모델의 고정 어휘로 작동하는 한계점과 이를 극복하기 위한 새로운 방법(단어를 부분 단어 단위의 시퀀스로 인코딩)을 소개합니다. NMT는 일반적으로 고정된 어휘로 작동하지만, 번역에는 개방 어휘 문제가 존재합니다. 즉, **모델이 학습할 때 정해진 어휘 사전 외에 존재하는 새로운 단어들에 대해 어떻게 처리할지에 대해 고민해야합니다.** 과거에는 사전에 존재하지 않은 단어(out-of-vocabulary word) 번역을 위해 "Backing off dictionary"를 활용하였습니다. 백오프(back-off) 사전은 OOV 단어를 다른 언어 모델이나 통계 정보를 활용하여 대체할 후보 단어를 찾는 방식입니다. 하지만, Back-off dictionary는 주로 고정된 패턴이나 미리 정의된 대안에 의존하기 때문에 사전에 충분한 예시가 없는 경우, 적절하거나 정확한 대안을 찾기 어려울 수 있습니다. 또한 백오프 사전을 구축하고 유지하는 것은 비용이 많이 들 수 있습니다. 따라서 본 논문에서는 **n-gram 모델과 바이트 페어 인코딩 압축 알고리즘을 기반으로 OOV에 대응하는 더 간단하고 효과적인 접근 방식을 소개**합니다.


# **Introduction**
**신경망 모델의 어휘는 일반적으로 30,000~50,000 단어로 제한**됩니다. 단어 수준 모델은 보이지 않는 단어를 번역하거나 생성할 수 없습니다. 따라서 번역 모델은 단어 수준 아래로 내려가는 메커니즘이 필요합니다. 우리는 희귀 단어를 서브워드 단위로 인코딩함으로써 백오프 모델없이 희귀 단어를 더 정확하게 번역하고 훈련 시에 보지 못한 새로운 단어를 생성할 수 있습니다.
본 논문의 주요 기여는 다음 두 가지입니다:
* 서브워드 단위를 통한 (희귀) 단어의 인코딩을 통해 *개방형 어휘 번역이* 가능한지를 보여줍니다. 이로써 서브 워드 구조가 큰 단어 구조를 사용하거나 백오프 사전 사용보다 더 간단하고 효과적임을 확인합니다.
    - *개방형 어휘 번역(Open-vocabulary neural machine translation): 번역 작업에서 고정된 어휘나 단어 목록에 제한받지 않고, 어휘의 범위를 개방적으로 다루는 기법을 나타냅니다.*
* 압축 알고리즘인 Byte Pair Encoding (BPE)를 단어 분할 작업에 적용합니다. BPE는 가변 길이의 문자 시퀀스로 구성된 (고정 크기의) 어휘사전을 통해 개방형 어휘를 표현할 수 있게 해줍니다. 이는 신경망 모델에 매우 적합한 단어 분할 전략입니다.

## **Neural Machine Translation**
신경망 기계 번역(Neural machine translation, NMT)은 일련의 단어 발생의 가능성을 예측하기 위해 인공 신경망을 사용하는 기계 번역 접근 방법으로, 순환 신경망을 사용한 인코더-디코더 네트워크로 구현됩니다. 논문에서는 Bahdanau 등의 구조를 따르지만 이 구조에만 특정되어있지는 않습니다.

## **Subword Translation**
이 논문의 주된 동기는 알려진 Subword(형태소나 음소) 등의 번역을 기반으로 새로운 단어도 번역될 수 있다는 것입니다. 저희의 독일어 훈련 데이터에서 (50,000개의 빈번한 유형에 속하지 않는)100개의 희귀 토큰에 대한 분석 결과, 대다수의 토큰이 영어로부터 더 작은 단위를 통해 번역 가능한 것으로 나타났습니다. 통계 기계 번역(SMT)에서는 알려지지 않은 단어의 번역이 중요한 연구 주제 중 하나였습니다. Phrase-based SMT에서 활용되는 segmentation 알고리즘들은 보수적으로 분리를 하는 경향이 있었습니다. 반면에, 해당 논문에서는 공격적인 분리를 통해 open-vocabulary 번역을 가능케하고 back-off dictionary에 의지하지 않도록하는 것을 목표로 합니다. 최적의 부분 단어 단위 선택은 작업에 따라 다를 수 있습니다. 논문은 텍스트를 효율적으로 압축하여 어휘를 학습할 수 있는 바이트 페어 인코딩(BPE)을 기반으로 한 분절 알고리즘을 제안합니다.

# **Byte Pair Encoding (BPE)**
바이트 페어 인코딩(BPE)은 (Gage, 1994)에 의해 제안된 간단한 데이터 압축 기술로, 연속된 **바이트 중에서 가장 빈번한 쌍을 새로운, 하나의 바이트로 대체**하는 기법입니다. 우리는 이 알고리즘을 단어 분절을 위해 적용합니다. 빈번한 문자 또는 문자 시퀀스를 병합합니다.

* [BPE 알고리즘]

 우선, 각 단어를 문자 시퀀스로 나타내며 특별한 ‘단어 종료 심볼’ '·'을 추가합니다. 이를 통해 우리는 번역 후에 원래의 토큰을 복원할 수 있습니다. 우리는 모든 심볼 쌍의 빈도수를 세어 가장 빈번한 쌍('A', 'B')을 새로운 심볼 'AB'로 대체합니다. 최종 심볼 어휘 크기는 초기 어휘 크기와 병합 작업 수로 이루어져 있으며, 후자는 알고리즘의 유일한 하이퍼파라미터입니다.

In [ ]:
import re, collections
def get_stats(vocab):
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs


def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

vocab = {'l o w </w>' : 5, 'l o w e r </w>' : 2, 'n e w e s t </w>':6,'w i d e s t </w>':3}
num_merges = 10

for i in range(num_merges):
    pairs = get_stats(vocab)
    best = max(pairs, key=pairs.get)
    # key=paris.get은 딕셔너리의 value를 기준으로 삼겠다는 의미
    # 빈도수가 같으면 딕셔너리중 먼저 나온 값이 반환됨.
    # 이 순서로 결과가 바뀔 수도 있기는 할 것이여!
    vocab = merge_vocab(best, vocab)
    print(f'pairs: {pairs}')
    print(f'bset: {best}\n')

print(f'최종 vocab: {vocab}')

pairs: defaultdict(<class 'int'>, {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 8, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('e', 's'): 9, ('s', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'e'): 3})
bset: ('e', 's')

pairs: defaultdict(<class 'int'>, {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('w', 'es'): 6, ('es', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'es'): 3})
bset: ('es', 't')

pairs: defaultdict(<class 'int'>, {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('w', 'est'): 6, ('est', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est'): 3})
bset: ('est', '</w>')

pairs: defaultdict(<class 'int'>, {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('w', 'est</w>'): 6,

## 논문 실험
논문에서 테스트는 두가지 다른 방법으로 BPE를 적용하여 진행됩니다. 소스 어휘에 대한 것과 타겟 어휘에 대한 것 두 가지 독립적인 인코딩을 학습하거나 두 어휘의 합집합에 대한 인코딩을 학습하는 것(이를 "합동 BPE"라고 함)입니다. BPE를 독립적으로 적용하면 텍스트와 어휘의 사이즈를 더 작게 만들 수 있다는 장점이 있는 방면, 동일한 이름이 두 언어에서 다르게 분절될 수 있으며, 이는 신경 모델이 부분 단어 단위 간의 매핑을 학습하기를 어렵게 만듭니다. 서로 다른 알파벳을 가진 영어와 러시아어 간의 분절 일관성을 높이기 위해 우리는 러시아어 어휘를 라틴 문자로 ISO-9로 음역하고, 합동 BPE 인코딩을 학습한 후에 BPE 병합 작업을 키릴 문자로 다시 음역하여 러시아어 교육 텍스트에 적용합니다.

# **평가**

논문에서는 다음 질문에 대한 답을 찾는 것을 목표로 합니다.
- **신경 기계 번역에서 부분 단어 단위를 사용하여 희귀하거나 이전에 보지 못한 단어의 번역을 개선할 수 있는가?**
- **어휘 크기, 텍스트 크기 및 번역 품질 측면에서 부분 단어 단위의 분절 중 어떤 것이 가장 성능이 우수한가?**

실험은 영어에서 독일어, 영어에서 러시아로 번역을 실행합니다. 결과는 BLEU (mteval-v13a.pl) 및 CHRF3 (Popovic, 2015)를 사용하여 보고합니다. CHRF3는 문자 n-gram F3 스코어로, 특히 영어 번역에 대한 인간 판단과 잘 상관된 것으로 확인되었습니다 (Stanojevic et al., 2015). BLEU는 기계 번역 결과와 사람이 직접 번역한 결과가 얼마나 유사한지 비교하여 번역에 대한 성능을 측정하는 방법입니다. 또한 논문의 **주요 논지는 희귀하거나 이전에 보지 못한 단어의 번역과 관련이 있기 때문에, 클리핑된 유니그램 정밀도와 리콜의 조화 평균으로 계산하는 유니그램 F1을 사용**하여 이를 측정합니다.


# **서브워드 통계**
번역 품질 외에도 테스트의 주요 목표는 컴팩트한 고정 크기의 서브워드 어휘를 통해 개방적 어휘를 표현하고 효율적인 훈련 및 디코딩을 가능하게 하는 것입니다. 기본이 되는 segmentation은 character n-gram입니다. n-gram은 단어를 n개의 글자로 끊어서 이를 하나의 토큰으로 간주합니다.
* *n-gram을 사용할 때는 n이 1일 때는 유니그램(unigram), 2일 때는 바이그램(bigram), 3일 때는 트라이그램(trigram)이라고 명명하고 n이 4 이상일 때는 gram 앞에 그대로 숫자를 붙여서 명명합니다.*

논문에서는 이러한 기술들이 어휘 크기를 어느 정도로만 감소시키고 알려지지 않은 단어 문제를 해결하지 못하며, 따라서 백오프 사전 없는 개방적 어휘 번역 목표에는 부적합하다고 주장합니다. BPE는 가변 길이의 단위의 유닛으로 작동하여 더 적은 토큰 수로 개방적 어휘의 목표를 충족시킵니다. Table1 표: BPE는 tokens(훈련 데이터 토큰 수)와 UNK(알려지지 않은 토큰 수)가 적음.

| Method                                   | Name Segmentation | Shortlist | Source | Target | Single Ens-8 | Single Ens-8 All | Rare OOV |
| ---------------------------------------- | ----------------- | --------- | ------ | ------ | ------------ | ---------------- | -------- |
| Syntax-based (Sennrich and Haddow, 2015) | 24.4              | -         | 55.3   | -      | 59.1         | 46.0             | 37.7     |
| WUnk                                     | -                 | -         | 300k   | 500k   | 20.6         | 22.8             | 47.2     |
| WDict                                    | -                 | -         | 300k   | 500k   | 22.0         | 24.2             | 50.5     |
| C2-50k char-bigram                       | 50k               | 60k       | 60k    | 22.8   | 25.3         | 51.9             | 53.5     |
| BPE-60k                                  | BPE               | 60k       | 60k    | 21.5   | 24.5         | 52.0             | 53.9     |
| BPE-J90k                                 | BPE (Joint)       | 90k       | 90k    | 22.8   | 24.7         | 51.7             | 54.1     |


---


| Method                               | Name Segmentation | Shortlist | Source | Target | Single Ens-8 | Single Ens-8 All | Rare OOV |
| ------------------------------------ | ----------------- | --------- | ------ | ------ | ------------ | ---------------- | -------- |
| Phrase-based (Haddow et al., 2015)   | 24.3              | -         | 53.8   | -      | 56.0         | 31.3             | 16.5     |
| WUnk                                 | -                 | -         | 300k   | 500k   | 18.8         | 22.4             | 46.5     |
| WDict                                | -                 | -         | 300k   | 500k   | 19.1         | 22.8             | 47.5     |
| C2-50k char-bigram                   | 50k               | 60k       | 60k    | 20.9   | 24.1         | 49.0             | 51.6     |
| BPE-60k                              | BPE               | 60k       | 60k    | 20.5   | 23.6         | 49.8             | 52.7     |
| BPE-J90k                             | BPE (Joint)       | 90k       | 100k   | 20.4   | 24.1         | 49.7             | 53.0     |


---


영어→독일어 번역 결과는 표 2에 나와 있으며, 영어→러시아어 결과는 표 3에 표시되어 있습니다. 기준이 되는 WDict는 단어 수준 모델이며 백오프 사전(back-off dictionary)을 사용합니다. OOV(Out-Of-Vocabulary) 단어의 경우, 영어→독일어의 경우 알파벳이 같기 때문에 알려지지 않은 단어를 복사하는 것이 잘 작동합니다. 따라서 OOV 단어에 대한 유니그램 F1을 향상시키지만, 영어→러시아어의 경우 백오프 사전이 이름을 전사할 수 없기 때문에 향상이 더 작습니다. 알파벳이 다를 때(영어→러시아어), 서브워드 모델이 훨씬 더 효과적입니다. 유니그램 F1을 기준으로 특히 희귀한 단어에 대해 (EN→DE의 경우 36.8%→41.8%, EN→RU의 경우 26.5%→29.7%) 향상이 있습니다.

# **5.1 유니그램 정확도 (Unigram accuracy)**

실험의 주요 주장은 단어 수준의 NMT 모델에서 희귀 및 알려지지 않은 단어의 번역이 부족하며, **서브워드 모델이 이러한 단어 유형의 번역을 개선**한다는 것입니다.
백오프 사전이 생성하는 모든 OOV(Out-of-Vocabulary)는 일반적으로 이름과 같은 소스에서 복사된 단어입니다. 반면 서브워드 시스템은 합성어와 같이 새로운 단어를 생성하는 데 생산적으로 활용될 수 있습니다.
* **백오프 사전은 주로 소스에서 복사된 이름과 같은 단어를 생성하지만, 서브워드 시스템은 새로운 합성어 등을 형성할 수 있다는 점에 주목해야합니다.**


| System  | Sentence                                 |
|---------|------------------------------------------|
| Source  | health research institutes               |
| Reference | Gesundheitsforschungsinstitute           |
| WDict   | Forschungsinstitute                       |
| C2-50k  | Fo|rs|ch|un|gs|in|st|it|ut|io|ne|n        |
| BPE-60k | Gesundheits|forsch|ungsinstitu|ten        |
| BPE-J90k| Gesundheits|forsch|ungsin|stitute         |

| System  | Sentence                                 |
|---------|------------------------------------------|
| Source  | asinine situation                        |
| Reference | dumme Situation                          |
| WDict   | asinine situation → UNK → asinine        |
| C2-50k  | as|in|in|e situation → As|in|en|si|tu|at|io|n |
| BPE-60k | as|in|ine situation → A|in|line-|Situation |
| BPE-J90K| as|in|ine situation → As|in|in-|Situation  |

* Table 4: English→German translation example.
“|” marks subword boundaries.

---
| System  | Sentence                  |
|---------|---------------------------|
| Source  | Mirzayeva                 |
| Reference | Мирзаева (Mirzaeva)      |
| WDict   | Mirzayeva → UNK → Mirzayeva |
| C2-50k  | Mi|rz|ay|ev|a → Ми|рз|ае|ва (Mi|rz|ae|va) |
| BPE-60k | Mirz|ayeva → Мир|за|ева (Mir|za|eva)      |
| BPE-J90k| Mir|za|yeva → Мир|за|ева (Mir|za|eva)    |

| System  | Sentence                    |
|---------|-----------------------------|
| Source  | rakfisk                     |
| Reference | ракфиска (rakfiska)         |
| WDict   | rakfisk → UNK → rakfisk     |
| C2-50k  | ra|kf|is|k → ра|кф|ис|к (ra|kf|is|k) |
| BPE-60k | rak|f|isk → пра|ф|иск (pra|f|isk)  |
| BPE-J90k| rak|f|isk → рак|ф|иска (rak|f|iska) |
* Table 5: English→Russian translation examples.
“|” marks subword boundaries.


---



표 4는 영어에서 독일어로의 번역 예제 두 가지를 보여주며, 표 5는 영어에서 러시아어로의 번역 예제 두 가지를 보여줍니다. 기본 시스템은 모든 예제에서 단어를 삭제하거나, 번역되어야 할 소스 단어를 그대로 복사하는 등 실패를 보입니다. 반면 부분 단어 번역은 부분 단어 시스템이 과도하게 분리될 때 (research→Fo|rs|ch|un|g) 또는 분절이 형태소 경계와 일치하지 않을 때도 학습할 수 있는 능력을 보여줍니다. 시스템이 데이터 희소성 때문에 번역을 학습하지 못한 경우 일부 외래어에서 (예: asinine은 dumm으로 번역되어야 함) 잘못된 번역을 볼 수 있지만, 외래어기 때문에 그럴듯한 번역이 될 수 있습니다.
영어에서 러시아어로의 예제는 부분 단어 시스템이 음역을 할 수 있음을 보여줍니다. BPE-60k 시스템은 인코딩을 두 언어 쌍에 대해 일관되지 않게 처리합니다만, 합동 BPE 시스템 (BPE-J90k)의 분절은 일관된 결과를 보여줍니다.

# **Conclusion**
이 논문의 주요 기여는 신경 기계 번역 시스템이 **rare and unseen words를 서브워드 유닛의 연속으로 나타내어 개방 어휘 번역이 가능하다는 것을 보여준 것**입니다. BPE 분절 및 간단한 문자 바이그램 분절 모두에서 기준 모델 대비 성능 향상을 보여줍니다. 우리의 어휘 크기 선택은 어느 정도 임의적이며 주로 이전 연구와의 비교를 위한 것입니다. 미래 연구의 한 방향은 번역 작업에 대한 최적의 어휘 크기를 자동으로 학습하는 것이며, 이는 언어 쌍 및 교육 데이터 양에 따라 달라질 것으로 기대됩니다. 언어별 요인, 어휘 크기와 같은 상대적 효과는 다를 수 있겠지만, 논문에서는 부분 단어 분절이 대부분의 언어 쌍에 적합하며, 큰 NMT 어휘나 back-off 모델이 필요하지 않음을 보여줍니다.
